In [1]:
import os
import json
import psycopg2
from psycopg2 import sql, extras

In [2]:
## Database Credentials

cred_pth = os.path.join(os.getcwd(),'db_credentials_template.txt')

with open(cred_pth, 'r') as f:
    
    creds = f.readlines()[0].rstrip('\n').split(', ')

pg_connection_dict = dict(zip(['dbname', 'user', 'password', 'port', 'host'], creds))

In [16]:
interp_name = 'PM2_5_Kriging'

print(f'Please select your interpolation:\n\n{interp_name}\n\nAnd add one of these to the base url')

Please select your interpolation:

PM2_5_Kriging

And add one of these to the base url


In [17]:
def f(interp_name=interp_name):
        
        conn = psycopg2.connect(**pg_connection_dict)

        # Create json cursor
        cur = conn.cursor(cursor_factory = extras.RealDictCursor)

        # Get the example
        cmd = f"""SELECT json_build_object(
    'type', 'FeatureCollection',
    'features', json_agg(ST_AsGeoJSON({interp_name}.*)::json)
    ) FROM {interp_name};"""

        # If want with CRS 'crs', json_build_object('type', 'name', properties', json_build_object( 'name', ST_SRID(>

        cur.execute(cmd)
        #  cur.execute('SELECT ST_ASGeoJSON(geom) FROM example')

        conn.commit() # Committ command

        geojson = json.loads(json.dumps(cur.fetchall()))[0]["json_build_object"]

        # Close connection
        cur.close()
        conn.close()

        return geojson

In [19]:
f()

OperationalError: connection to server at "34.27.240.213", port 5432 failed: Connection timed out
	Is the server running on that host and accepting TCP/IP connections?


In [18]:
# Endpoint List

    
endpoint = {
            "route": "/"+interp_name,
            "view_func": f
            }


endpoint

# # Add to Flask App

# for endpoint in endpoint_list:
#     app.add_url_rule(endpoint['route'], view_func=endpoint['view_func'])

{'route': '/PM2_5_Kriging',
 'view_func': <function __main__.f(interp_name='PM2_5_Kriging')>}

# Iterate

In [14]:
interp_names = ['PM2_5_Kriging', 'Temp_Kriging', 'Elevation_Kriging', 
                'PM2_5_Temp_Errors', 'Elevation_Errors']

endpoint_list = []

for i, interp_name in enumerate(interp_names): # Table Names

    ## Define the functions iteratively
    ## See here
    ## https://stackoverflow.com/questions/3431676/creating-functions-or-lambdas-in-a-loop-or-comprehension
    
    def f(interp_name=interp_name):
        
        conn = psycopg2.connect(**pg_connection_dict)

        # Create json cursor
        cur = conn.cursor(cursor_factory = extras.RealDictCursor)

        # Get the example as a Geojson
        cmd = f"""SELECT json_build_object(
    'type', 'FeatureCollection',
    'features', json_agg(ST_AsGeoJSON({interp_name}.*)::json)
    ) FROM {interp_name};"""

        cur.execute(cmd) # Execute

        conn.commit() # Committ command

        geojson = json.loads(json.dumps(cur.fetchall()))[0]["json_build_object"] # Unpack the response...
        
        ## ^ That's a dictionary

        # Close connection
        cur.close()
        conn.close()

        return geojson
        
    # Iteratively format endpoint dictionary 
    # FLASK APP This hack from:
    # Oooh, I lost the link...
    
    endpoint = {
                "route": "/"+interp_name,
                "view_func": f
                }
    
    endpoint_list.append(endpoint)
    
endpoint_list

[{'route': '/PM2_5_Kriging',
  'view_func': <function __main__.f(interp_name='PM2_5_Kriging')>},
 {'route': '/Temp_Kriging',
  'view_func': <function __main__.f(interp_name='Temp_Kriging')>},
 {'route': '/Elevation_Kriging',
  'view_func': <function __main__.f(interp_name='Elevation_Kriging')>},
 {'route': '/PM2_5_Temp_Errors',
  'view_func': <function __main__.f(interp_name='PM2_5_Temp_Errors')>},
 {'route': '/Elevation_Errors',
  'view_func': <function __main__.f(interp_name='Elevation_Errors')>}]